# 01 – Fundamentos de NumPy y Pandas con Wine

Objetivo general


Analizar el dataset Wine (scikit‑learn) mediante operaciones básicas y avanzadas de NumPy y Pandas, generando los artefactos de datos necesarios para los siguientes notebooks (Visualización, ML clásico y DL), sin incluir visualizaciones en esta etapa.
  


# Sección 1 – Configuración del Entorno

Propósito: Asegurar un entorno reproducible en Google Colab.

**1.1Verificación de versión de Python**



In [ ]:
import sys
print("Versión de Python:", sys.version)



Versión de Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


**1.2 Comprobación de GPU disponible (opcional para Deep Learning)**


In [ ]:
import tensorflow as tf
print("GPU disponible (TensorFlow):", tf.config.list_physical_devices('GPU'))


GPU disponible (TensorFlow): []


**1.3 Instalación de librerías adicionales (solo si no están presentes)**



In [ ]:
!pip install -q seaborn plotly


**1.4 Importaciones para análisis y visualización**



In [ ]:
import numpy as np                     # cálculo numérico
import pandas as pd                    # manipulación de tablas
import matplotlib.pyplot as plt        # gráficos básicos
import seaborn as sns                  # gráficos estadísticos
from sklearn.datasets import load_wine # dataset Wine integrado
import warnings
warnings.filterwarnings('ignore')      # suprimir advertencias innecesarias

**1.5 Configuración de estilo global para gráficos**

In [ ]:
plt.style.use('default')               # estilo Matplotlib por defecto
sns.set_palette("husl")                # paleta de colores uniforme
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size']      = 12

print("✅ Entorno configurado correctamente")

✅ Entorno configurado correctamente


# Sección 2 – Carga y Preparación del Dataset


Propósito: Obtener los datos en dos formatos complementarios.

- En esta sección se importan los datos del conjunto *Wine* incluido en *scikit‑learn*, se almacenan en estructuras NumPy y en un *DataFrame* de **pandas**, y se realiza una primera inspección exploratoria.



In [ ]:
# 2.1 Carga en estructuras NumPy
# ------------------------------
from sklearn.datasets import load_wine

wine = load_wine(as_frame=False)
X_raw = wine.data       # Matriz de atributos (178 × 13)
y      = wine.target    # Vector objetivo (178,)


La celda anterior:

- **Extrae** el *dataset* en formato “bunch” (diccionario enriquecido) usando `load_wine`.
- **Asigna** la matriz de características a `X_raw` y el vector de clases a `y`.
- Conserva la forma original (178 muestras, 13 atributos) para futuras comparaciones con otras estructuras de datos.


In [ ]:
# 2.2 Conversión a DataFrame de pandas
# ------------------------------------
import pandas as pd

df = pd.DataFrame(X_raw, columns=wine.feature_names)  # crea DataFrame con nombres descriptivos
df["target"]       = y
df["target_name"]  = pd.Categorical.from_codes(y, wine.target_names)  # mapea índices a etiquetas


En esta celda se **transforma** la representación NumPy en un *DataFrame* enriquecido con:

1. **Columnas nominales** (`wine.feature_names`) que facilitan la interpretación.
2. **Variable `target`** (numérica) para modelado supervisado.
3. **Variable `target_name`** (categórica) que traduce los códigos enteros a etiquetas legibles (`class_0`, `class_1`, `class_2`).  
Esta dualidad numérico‑categórica resulta útil para métricas de evaluación y visualización.


In [ ]:
# 2.3 Inspección inicial de la estructura
# ---------------------------------------
print("Dimensiones del DataFrame:", df.shape)
display(df.head())  # muestra las primeras cinco filas


Dimensiones del DataFrame: (178, 15)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target,target_name
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0,class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0,class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0,class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0,class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0,class_0


#### Hallazgos preliminares  

- **Dimensiones**: (178, 15) — 178 registros y 15 columnas (13 atributos fisicoquímicos + `target` + `target_name`).  
- **Completitud**: no se detectan valores nulos en ninguna columna.  
- **Escalas heterogéneas**: variables como `proline` presentan rangos de magnitud considerablemente superiores a los de `nonflavanoid_phenols`, lo que anticipa la necesidad de normalización o estandarización en etapas posteriores.  
- **Representatividad de clases**: `class_1` ≈ 40 %, `class_0` ≈ 33 % y `class_2` ≈ 27 %; el leve desbalance aconseja emplear validación cruzada estratificada.  


# Sección 3 – Inspección Estadística con Pandas


*Propósito general: evaluar la calidad del *dataset*, caracterizar la distribución de las variables y detectar valores atípicos antes de cualquier modelado.*


In [ ]:
# 3.1 Tipos de datos y valores nulos
print("Tipos de datos:\n", df.dtypes, "\n")
print("Valores nulos por columna:\n", df.isnull().sum(), "\n")


Tipos de datos:
 alcohol                          float64
malic_acid                       float64
ash                              float64
alcalinity_of_ash                float64
magnesium                        float64
total_phenols                    float64
flavanoids                       float64
nonflavanoid_phenols             float64
proanthocyanins                  float64
color_intensity                  float64
hue                              float64
od280/od315_of_diluted_wines     float64
proline                          float64
target                             int64
target_name                     category
dtype: object 

Valores nulos por columna:
 alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity   

### 3.1 Tipos de datos y valores nulos  
- **Tipos de datos**  
  - Se identificaron **13 columnas `float64`** (atributos fisicoquímicos).  
  - Se registró **1 columna `int64`** correspondiente a la variable objetivo `target`.  
  - Se incluyó **1 columna categórica** (`target_name`) para la etiqueta legible de clase.  
- **Valores nulos**  
  - No se detectaron valores faltantes; las **15 columnas** presentaron **0 % de `NaN`**.  
- **Implicación**  
  - La completitud del conjunto de datos eliminó la necesidad de aplicar técnicas de imputación.  


In [ ]:
# 3.2 Estadísticas descriptivas + dispersión y forma
desc = df.describe().T
desc['coef_var'] = desc['std'] / desc['mean']
# Exclude non-numeric columns for skew and kurtosis calculations
numeric_cols = df.select_dtypes(include=np.number).columns
desc['skew']     = df[numeric_cols].skew()
desc['kurtosis'] = df[numeric_cols].kurtosis()
print("Estadísticas descriptivas y forma de distribución:")
display(desc)



Estadísticas descriptivas y forma de distribución:


,count,mean,std,min,25%,50%,75%,max,coef_var,skew,kurtosis
alcohol,178.0,13.000618,0.811827,11.03,12.3625,13.050,13.6775,14.83,0.062445,-0.051482,-0.852500
malic_acid,178.0,2.336348,1.117146,0.74,1.6025,1.865,3.0825,5.80,0.478159,1.039651,0.299207
ash,178.0,2.366517,0.274344,1.36,2.2100,2.360,2.5575,3.23,0.115927,-0.176699,1.143978
alcalinity_of_ash,178.0,19.494944,3.339564,10.60,17.2000,19.500,21.5000,30.00,0.171304,0.213047,0.487942
magnesium,178.0,99.741573,14.282484,70.00,88.0000,98.000,107.0000,162.00,0.143195,1.098191,2.104991
total_phenols,178.0,2.295112,0.625851,0.98,1.7425,2.355,2.8000,3.88,0.272689,0.086639,-0.835627
flavanoids,178.0,2.029270,0.998859,0.34,1.2050,2.135,2.8750,5.08,0.492226,0.025344,-0.880382
nonflavanoid_phenols,178.0,0.361854,0.124453,0.13,0.2700,0.340,0.4375,0.66,0.343933,0.450151,-0.637191
proanthocyanins,178.0,1.590899,0.572359,0.41,1.2500,1.555,1.9500,3.58,0.359771,0.517137,0.554649
color_intensity,178.0,5.058090,2.318286,1.28,3.2200,4.690,6.2000,13.00,0.458332,0.868585,0.381522


### 3.2 Estadística descriptiva, dispersión y forma de distribución  
- **Coeficiente de variación (CV ≥ 0.30)**  
  - Alta dispersión relativa en `flavanoids` (0.49), `malic_acid` (0.48), `color_intensity` (0.46) y `proline` (0.42).  
  - Se sugirió escalar y, de ser necesario, transformar (log o Box‑Cox) dichas variables.  
- **Asimetría significativa (|skew| > 1)**  
  - Sesgo a la derecha observado en `magnesium` (1.10) y `malic_acid` (1.04).  
  - Se recomendó evaluar transformaciones para aproximar la normalidad si se emplean métodos lineales.  
- **Curtosis elevada (> 1)**  
  - Colas ligeramente pesadas en `magnesium` (2.10) y, en menor grado, en `malic_acid` (0.30).  
  - Se planteó revisar el impacto en técnicas paramétricas.  
- **Baja dispersión (CV < 0.10)**  
  - `alcohol` mostró la menor variabilidad (0.06), indicando estabilidad numérica tras estandarización.  
- **Implicación principal**  
  - Las variables con alta dispersión y asimetría se priorizaron para escalado y posibles transformaciones de potencia.  

In [ ]:
# 3.3 Detección de outliers (IQR)
iqr   = desc['75%'] - desc['25%']
lower = desc['25%'] - 1.5 * iqr
upper = desc['75%'] + 1.5 * iqr
outlier_counts = {
    col: int(((df[col] < lower[col]) | (df[col] > upper[col])).sum())
    for col in wine.feature_names
}
print("Outliers por variable (IQR):")
display(pd.Series(outlier_counts))

Outliers por variable (IQR):


,0
alcohol,0
malic_acid,3
ash,3
alcalinity_of_ash,4
magnesium,4
total_phenols,0
flavanoids,0
nonflavanoid_phenols,0
proanthocyanins,2
color_intensity,4


### 3.3 Detección de valores atípicos (criterio IQR)  
- **Variables con 4 outliers**  
  - `alcalinity_of_ash`, `magnesium`, `color_intensity`.  
  - Necesidad de inspección individual y posible winsorización.  
- **Variables con 3 outliers**  
  - `malic_acid`, `ash`.  
  - Outliers moderados; se recomendó validar con métodos robustos.  
- **Variables con 2 outliers**  
  - `proanthocyanins`.  
  - Riesgo limitado, pero se aconsejó revisión visual complementaria.  
- **Variables con 1 outlier**  
  - `hue`.  
  - Impacto mínimo sobre la varianza global.  
- **Variables sin outliers (bajo criterio IQR)**  
  - `alcohol`, `total_phenols`, `flavanoids`, `nonflavanoid_phenols`, `od280/od315_of_diluted_wines`, `proline`.  
- **Implicación general**  
  - El nivel de atipicidad fue en su mayoría manejable; se sugirió un tratamiento focalizado en las variables con cuatro outliers para evitar distorsiones en modelos paramétricos.  



##**Recomendación integrada**  
> - Aplicar **escalado z‑score** a todas las variables numéricas.  
> - Evaluar **transformaciones logarítmicas** o Box‑Cox en las variables con alta dispersión y asimetría.  
> - Tratar outliers (winsorización o métodos robustos) en `alcalinity_of_ash`, `magnesium` y `color_intensity`.  
> - Emplear **validación cruzada estratificada** para preservar el ligero desbalance de clases durante el modelado.  

# Sección 4 – Separación de Variables Explicativas (X) y Etiqueta (y)



In [ ]:
# 4.1 Extracción de X e y
X = df.drop(columns=['target','target_name']).to_numpy()
y = df['target'].to_numpy()



### 4.1 Extracción de matrices  
- **Matriz `X`**  
  - Se creó eliminando las columnas `target` y `target_name` del *DataFrame* y convirtiendo el resultado a *NumPy array*.  
  - Contiene exclusivamente las **13 variables fisicoquímicas** (predictoras).  
- **Vector `y`**  
  - Se extrajo la columna `target` en formato *NumPy* de dimensión unidimensional.  
  - Representa la **clase numérica** asociada a cada registro.  

In [ ]:
# 4.2 Verificación de dimensiones
print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

Forma de X: (178, 13)
Forma de y: (178,)


### 4.2 Verificación de dimensiones  
- **Forma de `X`**: (178, 13)  
  - 178 observaciones × 13 atributos predictivos.  
- **Forma de `y`**: (178,)  
  - 178 etiquetas correspondientes, sin discrepancias en el recuento de muestras.  

> *Implicación:* la consistencia de dimensiones confirma que el conjunto de datos está listo para ser dividido en entrenamiento y prueba, así como para cualquier procedimiento de escalado o validación cruzada posterior.


# Sección 5 – Ejercicios de Manipulación con NumPy


Propósito: Practicar operaciones en arrays y generar artefactos intermedios.

---



In [ ]:
# 5.1 Información general
print("Dimensiones:", X.ndim)
print("Forma:", X.shape)
print("Total elementos:", X.size)
print("Bytes/elemento:", X.itemsize, "\n")

Dimensiones: 2
Forma: (178, 13)
Total elementos: 2314
Bytes/elemento: 8 



### 5.1 Información general del arreglo `X`
- **Dimensiones (`ndim`)**: 2 → matriz bidimensional.  
- **Forma (`shape`)**: (178, 13) → 178 registros × 13 atributos.  
- **Número total de elementos (`size`)**: 2 314.  
- **Tamaño de cada elemento (`itemsize`)**: 8 bytes (tipo `float64`).  

In [ ]:
# 5.2 Primeras/Últimas filas
print("Primeras 3 muestras:\n", X[:3], "\n")
print("Últimas 3 muestras:\n", X[-3:], "\n")

Primeras 3 muestras:
 [[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
  2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 1.120e+01 1.000e+02 2.650e+00 2.760e+00
  2.600e-01 1.280e+00 4.380e+00 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 1.860e+01 1.010e+02 2.800e+00 3.240e+00
  3.000e-01 2.810e+00 5.680e+00 1.030e+00 3.170e+00 1.185e+03]] 

Últimas 3 muestras:
 [[1.327e+01 4.280e+00 2.260e+00 2.000e+01 1.200e+02 1.590e+00 6.900e-01
  4.300e-01 1.350e+00 1.020e+01 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 2.000e+01 1.200e+02 1.650e+00 6.800e-01
  5.300e-01 1.460e+00 9.300e+00 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 2.450e+01 9.600e+01 2.050e+00 7.600e-01
  5.600e-01 1.350e+00 9.200e+00 6.100e-01 1.600e+00 5.600e+02]] 



### 5.2 Visualización de registros extremos
- **Primeras 3 muestras**: permiten verificar la correcta carga de los valores originales.  
- **Últimas 3 muestras**: confirman la integridad de índices y el orden del *DataFrame* tras la conversión.  


In [ ]:
# 5.3 Indexación y slicing
print("Muestra 0:", X[0])
print("Alcohol (10 primeras):", X[:10,0])
print("Subset 50×5:", X[:50,:5].shape, "\n")

Muestra 0: [1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
Alcohol (10 primeras): [14.23 13.2  13.16 14.37 13.24 14.2  14.39 14.06 14.83 13.86]
Subset 50×5: (50, 5) 



### 5.3 Indexación y *slicing*
- **Muestra 0**: acceso a todos los atributos de la primera observación.  
- **Columna `alcohol` (primeras 10 filas)**: demostración de *slicing* por filas y columnas.  
- **Submatriz 50 × 5**: extracción de las 50 primeras filas y 5 primeras columnas → forma (50, 5).  



In [ ]:

# 5.4 Estadísticas vectorizadas
medias = np.mean(X, axis=0)
stds   = np.std(X, axis=0)
print("Media (5 primeras):", medias[:5])
print("Std   (5 primeras):", stds[:5])
print("Media por muestra (10 primeras):", np.mean(X,axis=1)[:10], "\n")

Media (5 primeras): [13.00061798  2.33634831  2.36651685 19.49494382 99.74157303]
Std   (5 primeras): [ 0.80954291  1.11400363  0.27357229  3.33016976 14.24230767]
Media por muestra (10 primeras): [ 95.76923077  91.85384615 103.21692308 126.96076923  69.89923077
 124.24846154 110.49692308 112.98307692  91.67846154  92.03538462] 



### 5.4 Estadísticas vectorizadas
- **Media global** (cinco primeras variables):  
  - `[13.00, 2.34, 2.37, 19.49, 99.74]`.  
- **Desviación estándar correspondiente**:  
  - `[0.81, 1.11, 0.27, 3.33, 14.24]`.  
- **Media por muestra** (primeras 10 observaciones): valores promedios cercanos a 90–127 evidencian la disparidad de escalas.  


In [ ]:

# 5.5 Filtrado y búsqueda
mask = X[:,0] > 14
print("Vinos con alcohol >14:", mask.sum())
print("Índices alcohol>14:", np.where(mask)[0][:5])
print("Máx alcohol:", X[np.argmax(X[:,0]),0])
print("Mín alcohol:", X[np.argmin(X[:,0]),0], "\n")

Vinos con alcohol >14: 22
Índices alcohol>14: [0 3 5 6 7]
Máx alcohol: 14.83
Mín alcohol: 11.03 



### 5.5 Filtrado y búsqueda
- **Vinos con `alcohol > 14`**: 22 registros.  
- **Primeros índices con dicha condición**: `[0, 3, 5, 6, 7]`.  
- **Valor máximo de alcohol**: 14.83 %.  
- **Valor mínimo de alcohol**: 11.03 %.  

In [ ]:
# 5.6 Reshape y flatten
print("Flatten:", X.flatten().shape)
print("Reshape 178×?:", X.reshape(178,-1).shape)
print("Transpuesto:", X.T.shape, "\n")

Flatten: (2314,)
Reshape 178×?: (178, 13)
Transpuesto: (13, 178) 



### 5.6 Transformaciones de forma
- **Vector plano (`flatten`)**: forma (2 314,) → linealización completa.  
- **Reestructuración `reshape(178, -1)`**: recuperación de la matriz original (178, 13).  
- **Transposición (`X.T`)**: forma (13, 178) → facilita operaciones por variable.  


In [ ]:
# 5.7 Concatenación y división
X1, X2, X3 = X[:60], X[60:120], X[120:]
print("Bloques:", X1.shape, X2.shape, X3.shape)
print("Reconstruido == original?", np.array_equal(X, np.concatenate([X1,X2,X3],axis=0)), "\n")


Bloques: (60, 13) (60, 13) (58, 13)
Reconstruido == original? True 



### 5.7 Concatenación y división
- **Bloques creados**:  
  - `X1` → (60, 13)  
  - `X2` → (60, 13)  
  - `X3` → (58, 13)  
- **Reconstrucción**: `np.concatenate([X1, X2, X3])` ≡ `X` → **verdadero**, se preservó el orden original.  


In [ ]:
# 5.8 Máscaras booleanas por clase
for cls in [0,1,2]:
    m = (y==cls)
    print(f"Clase {cls}: {m.sum()} muestras, media alcohol={X[m,0].mean():.2f}")
print()

Clase 0: 59 muestras, media alcohol=13.74
Clase 1: 71 muestras, media alcohol=12.28
Clase 2: 48 muestras, media alcohol=13.15



### 5.8 Máscaras booleanas por clase
- **Clase 0**: 59 muestras, media de alcohol = 13.74 %.  
- **Clase 1**: 71 muestras, media de alcohol = 12.28 %.  
- **Clase 2**: 48 muestras, media de alcohol = 13.15 %.  
- *Implicación*: la variable `alcohol` podría ayudar a discriminar entre clases 0 y 1.  


In [ ]:
# 5.9 Normalización Min‑Max
X_norm = (X - X.min(0)) / (X.max(0) - X.min(0))
print("Norm min/max:", X_norm.min(), "/", X_norm.max(), "\n")

Norm min/max: 0.0 / 1.0 



### 5.9 Normalización *Min‑Max*
- **Rango obtenido**: `[0.0, 1.0]` para todos los atributos.  
- *Implicación*: adecuado para algoritmos basados en distancias (p. ej., *k*-NN).  

In [ ]:


# 5.10 Estandarización Z‑score
X_std = (X - medias)/stds
print("Std mean/std:", X_std.mean().round(2), "/", X_std.std().round(2), "\n")

Std mean/std: 0.0 / 1.0 



### 5.10 Estandarización *Z‑score*
- **Media global tras estandarizar**: 0.00.  
- **Desviación estándar global**: 1.00.  
- *Implicación*: idónea para modelos lineales y redes neuronales.  

In [ ]:
# 5.11 Conversión a tensor (DL)
import tensorflow as tf
wine_tensor = tf.convert_to_tensor(X_norm, dtype=tf.float32)
print("Tensor DL shape:", wine_tensor.shape, "\n")

Tensor DL shape: (178, 13) 



### 5.11 Conversión a tensor para *Deep Learning*
- **Forma del tensor**: (178, 13).  
- **Tipo de dato**: `tf.float32`.  
- *Implicación*: el arreglo está listo para ser usado como entrada en modelos de TensorFlow/Keras.  


In [ ]:
# 5.12 Exportación de resultados
import os
os.makedirs('data/exercises', exist_ok=True)

# a) Estadísticas → CSV
pd.DataFrame({'feature': wine.feature_names,
              'mean': medias, 'std': stds})\
  .to_csv('data/exercises/feature_stats.csv', index=False)

# b) Arrays procesados → NPZ
np.savez('data/exercises/processed_arrays.npz',
         X_raw=X, X_norm=X_norm, X_std=X_std, y=y)

# c) Correlación → Parquet
corr = np.corrcoef(X.T)
pd.DataFrame(corr,
             index=wine.feature_names,
             columns=wine.feature_names)\
  .to_parquet('data/exercises/correlation_matrix.parquet')

### 5.12 Exportación de artefactos
- **`feature_stats.csv`**: medias y desviaciones estándar por atributo.  
- **`processed_arrays.npz`**: contiene `X_raw`, `X_norm`, `X_std` y `y`.  
- **`correlation_matrix.parquet`**: matriz de correlación 13 × 13 en formato optimizado para análisis posterior.  

# Sección 6 – Preparación de Artefactos


| Archivo                                     | Destino Notebook                |
| ------------------------------------------- | ------------------------------- |
| `data/wine_raw.parquet`                     | 02\_Visualizacion\_Datos        |
| `data/wine_ml_ready.npz`                    | 03\_Machine\_Learning\_Basico   |
| `data/wine_tfdataset/`                      | 04\_Deep\_Learning\_Intro       |
| `data/exercises/feature_stats.csv`          | Estadísticas de ejercicios      |
| `data/exercises/processed_arrays.npz`       | Arrays procesados de ejercicios |
| `data/exercises/correlation_matrix.parquet` | Matriz de correlación           |


In [ ]:
# 6.1 Guardar DataFrame original
df.to_parquet('data/wine_raw.parquet', index=False)

# 6.2 NPZ para ML clásico
np.savez('data/wine_ml_ready.npz',
         X_raw=X, X_norm=X_norm, X_std=X_std, y=y)

# 6.3 tf.data.Dataset para DL
tfdataset = tf.data.Dataset.from_tensor_slices((X_norm, y))
tf.data.experimental.save(tfdataset, 'data/wine_tfdataset')


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


# Sección 7 – Conclusiones



1. **Validación e integridad del *dataset***  
   - Se constató la ausencia total de valores faltantes en las **178 observaciones** y **15 columnas**, así como la correcta tipificación de los datos (13 `float64`, 1 `int64`, 1 categórica).  

2. **Exploración estadística exhaustiva**  
   - El análisis de dispersión (coeficiente de variación), asimetría (*skew*), curtosis y detección de atípicos (criterio IQR) brindó una caracterización detallada de cada variable, fundamentando futuras estrategias de escalado y tratamiento de outliers.  

3. **Ejercicios de manipulación NumPy y generación de artefactos**  
   - Se completaron **12 operaciones clave**—slicing, filtrado booleano, normalización *Min‑Max*, estandarización *Z‑score*, *reshape*, concatenación y conversión a tensor—consolidando competencias en manejo de arrays.  
   - Se exportaron artefactos reproducibles:  
     - `feature_stats.csv` (estadísticas descriptivas),  
     - `processed_arrays.npz` (matrices crudas y transformadas) y  
     - `correlation_matrix.parquet` (matriz de correlación 13 × 13).  

4. **Preparación para etapas posteriores**  
   - Los conjuntos `X_norm`, `X_std` y el tensor `wine_tensor` constituyen insumos listos para **visualización exploratoria**, **modelos de aprendizaje automático clásico** y **redes neuronales profundas**, respectivamente, garantizando continuidad metodológica en los notebooks sucesivos.  